In [11]:
#our first version

from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import time

# Setup Firefox WebDriver
# Make sure you have geckodriver installed and available in your PATH
service = Service()
driver = webdriver.Firefox(service=service)

# Open the property page
url = "https://immovlan.be/en/detail/villa/for-sale/9051/sint-denijs-westrem/rbu55821"
driver.get(url)

###########

# ---- Handle cookie consent automatically ----
try:
    # Wait for the "Agree and close" button to be clickable
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
    )
    accept_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("No cookie popup found or could not click the accept button:", e)
    
# ---- End cookie handling ----

#################
time.sleep(10)  # Wait for the page to fully load

# TERRACE: 1 = Yes, 0 = No, None if missing 
try:
    terrace_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Terrace')]/following-sibling::p").text
    terrace = 1 if "Yes" in terrace_text else 0
except:
    terrace = None

# GARDEN: 1 = Yes, 0 = No
try:
    garden_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Garden')]/following-sibling::p").text
    garden = 1 if "Yes" in garden_text else 0
except:
    garden = 0

# GARDEN AREA (m²): integer or None
try:
    garden_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Surface garden')]/following-sibling::p").text
    garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
except:
    garden_area = None

# NUMBER OF FACADES: integer or None 
try:
    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
    facades = int(facades_text)
except:
    facades = None

# SWIMMING POOL: 1 = Yes, 0 = No
try:
    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
    swimming_pool = 1 if "Yes" in pool_text else 0
except:
    swimming_pool = 0

# STATE OF BUILDING: text or None
try:
    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
except:
    state = None

# BUILD YEAR: integer or None 
try:
    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
    build_year = int(build_year_text)
except:
    build_year = None

# NUMBER OF ROOMS: integer or None
try:
    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
    number_rooms = int(number_rooms_text)
except:
    number_rooms = None

# LIVING AREA (m²): integer or None
try:
    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
except:
    living_area = None

# EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
# EQUIPPED KITCHEN: 1 = yes, 0 = no, None if missing
try:
    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
        equipped_kitchen = 1
    elif "not" in kitchen_text.lower():      # text containing "not"
        equipped_kitchen = 0
    else:
        equipped_kitchen = None
except:
    equipped_kitchen = None

# FURNISHED: 1 = Yes, 0 = No, None if missing
try:
    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
    if "yes" in furnished_text.lower():
        furnished = 1
    elif "no" in furnished_text.lower():
        furnished = 0
    else:
        furnished = None
except:
    furnished = None

# # OPEN FIRE: 1 = Yes, 0 = No, None if missing 
# try:
#     open_fire_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Open fire')]/following-sibling::p").text
#     if "Yes" in open_fire_text:
#         open_fire = 1
#     elif "No" in open_fire_text:
#         open_fire = 0
#     else:
#         open_fire = None
# except:
#     open_fire = None

# Print results 
print("Terrace:", terrace)
print("Garden presence:", garden)
print("Garden area:", garden_area)
print("Number of facades:", facades)
print("Swimming pool:", swimming_pool)
print("State of building:", state)
print("Build year:", build_year)
print("Number of rooms:", number_rooms)
print("Living area (m²):", living_area)
print("Equipped kitchen:", equipped_kitchen)
print("Furnished:", furnished)
#print("Open fire:", open_fire)

# Close browser 
#driver.quit()

Cookies accepted.
Terrace: 1
Garden presence: 1
Garden area: 470
Number of facades: 4
Swimming pool: 0
State of building: Excellent
Build year: 1954
Number of rooms: 3
Living area (m²): 185
Equipped kitchen: 1
Furnished: 0


In [ ]:
#To make it a function
#  


In [9]:

#Welde version with 2 more categories for test (lets work on this):

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added


# URL to scrape
urls = [
    "https://immovlan.be/en/detail/villa/for-sale/9051/sint-denijs-westrem/rbu55821"
]

# Firefox options (headless)
options = Options()
options.add_argument("--headless")

# Loop through URLs
for url in urls:
    driver = None
    try:
        # Launch Firefox
        service = Service(GeckoDriverManager().install())
        driver = webdriver.Firefox(service=service, options=options)
        driver.get(url)

        

        # ---- Handle cookie consent automatically ----
        try:
            # Wait for the "Agree and close" button to be clickable
            accept_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
            )
            accept_button.click()
            print("Cookies accepted.")
        except Exception as e:
            print("No cookie popup found or could not click the accept button:", e)
    
    # ---- End cookie handling ----

        try:
            terrace_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('Terrace') + p")
            terrace_text = terrace_tag.text.strip()
            terrace = 1 if "yes" in terrace_text.lower() else 0
        except:
            terrace = None 




        # GARDEN: 1 = Yes, 0 = No
        try:
            garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
            garden = 0  # default
            for block in garden_blocks:
                try:
                    h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                    if "garden" == h4_text:  # exact match for Garden
                        garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                        garden = 1 if "yes" in garden_text.lower() else 0
                        break
                except:
                    continue
        except:
            garden = 0

            



            # GARDEN AREA (m²): integer or None
            try:
                garden_area = None
                for block in garden_blocks:
                    try:
                        h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                        if "surface garden" in h4_text:
                            garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                            garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                            break
                    except:
                        continue
            except:
                garden_area = None

           

        # -----------------------
        # Extract property ID
        # -----------------------
        try:
            property_id = url.split('/')[-1]
        except:
            property_id = None

        # -----------------------
        # Extract postal code & locality
        # -----------------------
        try:
            city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
            city_text = city_tag.text.strip()
            postal_code = city_text.split()[0]
            locality_name = " ".join(city_text.split()[1:])
        except:
            postal_code = None
            locality_name = None

        # -----------------------
        # Extract price
        # -----------------------
        try:
            price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
            price = price_tag.text.strip()
        except:
            price = None

        # -----------------------
        # Extract property type & subtype
        # -----------------------
        try:
            type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
            type_words = type_tag.text.strip().split()
            property_type = type_words[0]
            #subtype = " ".join(type_words[1:]) if len(type_words) > 1 else ""
        except:
            property_type = None
            #subtype = None
            

        # -----------------------
        # Print results
        # -----------------------
        print("Terrace:", terrace)
        print("Garden:", garden)
        print("Garden area (m²):", garden_area)
        
        print("Property ID:", property_id)
        print("Postal Code:", postal_code)
        print("Locality:", locality_name)
        print("Price:", price)
        print("Type:", property_type)
        #print("Subtype:", subtype)
        print("-" * 40)

    except:
        print("Error loading page:", url)
    finally:
        if driver:
            driver.quit()

Cookies accepted.
Terrace: None
Garden: 1
Garden area (m²): 470
Property ID: rbu55821
Postal Code: 9051
Locality: Sint-Denijs-Westrem
Price: 860 000 €
Type: Villa
----------------------------------------


In [ ]:

#the cookie based on the advice
# ---- Handle cookie consent automatically ----
try:
    # Wait for the "Agree and close" button to be clickable
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
    )
    accept_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("No cookie popup found or could not click the accept button:", e)
    
# ---- End cookie handling ----

In [4]:

#our working code :-)))))


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_properties(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["URL", "Terrace", "Garden", "Garden area (m²)", "Property ID", "Postal Code", "Locality", "Price", "Type"])

        # Loop through URLs
        for url in urls:
            driver = None
            try:
                # Launch Firefox
                service = Service(GeckoDriverManager().install())
                driver = webdriver.Firefox(service=service, options=options)
                driver.get(url)

                # ---- Handle cookie consent automatically ----
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                ##################################
                
               # NUMBER OF FACADES: integer or None
                # try:
                #     # Selects the <p> element immediately following an <h4> inside the same parent <div>
                #     facades_tag = driver.find_element(By.CSS_SELECTOR, "div > h4 + p")
                #     facades_text = facades_tag.text.strip()
                #     facades = int(facades_text)
                # except:
                #     facades = None

                ##################################
                # # NUMBER OF FACADES: integer or None 
                # try:
                #     facades_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('facades') + p")
                #     facades_text = facades_tag.text.strip()
                #     facades = int(facades_text)
                #     print(facades_text)
                # except Exception as e:
                #     facades = None
                #     print(f"exception{e}")
                

                  #############################  :-) 

                # NUMBER OF FACADES: integer or None 
                try:
                    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
                    facades = int(facades_text)
                except:
                    facades = None


                    ################################  :-)
                # SWIMMING POOL: 1 = Yes, 0 = No
                try:
                    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
                    swimming_pool = 1 if "Yes" in pool_text else 0
                except:
                    swimming_pool = 0


                ###########################"" ADDING MULTIPLE ITESM
                # STATE OF BUILDING: text or None
                try:
                    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
                except:
                    state = None

                # BUILD YEAR: integer or None 
                try:
                    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
                    build_year = int(build_year_text)
                except:
                    build_year = None

                # NUMBER OF ROOMS: integer or None
                try:
                    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
                    number_rooms = int(number_rooms_text)
                except:
                    number_rooms = None

                    # LIVING AREA (m²): integer or None
                try:
                    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
                    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
                except:
                    living_area = None

                # EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
            
                try:
                    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
                    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
                        equipped_kitchen = 1
                    elif "not" in kitchen_text.lower():      # text containing "not"
                        equipped_kitchen = 0
                    else:
                        equipped_kitchen = None
                except:
                    equipped_kitchen = None

                # FURNISHED: 1 = Yes, 0 = No, None if missing
                try:
                    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
                    if "yes" in furnished_text.lower():
                        furnished = 1
                    elif "no" in furnished_text.lower():
                        furnished = 0
                    else:
                        furnished = None
                except:
                        furnished = None

                ###########################"" MULTIPLE ITEMS ADDED ABOVE  
                
                try:
                    terrace_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('Terrace') + p")
                    terrace_text = terrace_tag.text.strip()
                    terrace = 1 if "yes" in terrace_text.lower() else 0
                except:
                    terrace = None 

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Terrace:", terrace)
                print("Garden presence:", garden)
                print("Garden area:", garden_area)
                print("Number of facades:", facades)
                print("Swimming pool:", swimming_pool)
                print("State of building:", state)
                print("Build year:", build_year)
                print("Number of rooms:", number_rooms)
                print("Living area (m²):", living_area)
                print("Equipped kitchen:", equipped_kitchen)
                print("Furnished:", furnished)
                print("Property ID:", property_id)
                print("Postal Code:", postal_code)
                print("Locality:", locality_name)
                print("Price:", price)
                print("Type:", property_type)
                print("-" * 40)

                # Write row to CSV file
                writer.writerow([url, terrace, garden, garden_area, property_id, postal_code, locality_name, price, property_type])

            except:
                print("Error loading page:", url)
            finally:
                if driver:
                    driver.quit()


# Example use:
urls = [
    "https://immovlan.be/en/detail/villa/for-sale/9051/sint-denijs-westrem/rbu55821"] 

scrape_properties(urls)  # <-- added function call


Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 470
Number of facades: 4
Swimming pool: 0
State of building: Excellent
Build year: 1954
Number of rooms: 3
Living area (m²): 185
Equipped kitchen: 1
Furnished: 0
Property ID: rbu55821
Postal Code: 9051
Locality: Sint-Denijs-Westrem
Price: 860 000 €
Type: Villa
----------------------------------------


In [ ]:
#working on finding the urls

<a href="https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu59338" title="Apartment for sale Gent (RBU59338)" rel="nofollow">
                                <div class="media-pic-wrapper">
                                    <img class="centered-and-cropped lazyloaded" alt="Apartment for sale Gent (RBU59338)" title="Apartment for sale Gent (RBU59338)" src="https://api-image.immovlan.be/v1/property/RBU59338/thumbnail-webp/Medium?h=280" data-src="https://api-image.immovlan.be/v1/property/RBU59338/thumbnail-webp/Medium?h=280" itemprop="photo" content="https://api-image.immovlan.be/v1/property/RBU59338/thumbnail-webp/Medium?h=280">

                                        <div class="ribbon ribbon-top-left">
                                            <span>Best of</span>
                                        </div>

                                            <div class="energy-consumption epc-water-mark FlandersD"></div>
                                </div>
                            </a>


<a href="https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu59338" title="Apartment for sale Gent (RBU59338)" rel="nofollow">
                                <div class="media-pic-wrapper">
                                    <img class="centered-and-cropped lazyloaded" alt="Apartment for sale Gent (RBU59338)" title="Apartment for sale Gent (RBU59338)" src="https://api-image.immovlan.be/v1/property/RBU59338/thumbnail-webp/Medium?h=280" data-src="https://api-image.immovlan.be/v1/property/RBU59338/thumbnail-webp/Medium?h=280" itemprop="photo" content="https://api-image.immovlan.be/v1/property/RBU59338/thumbnail-webp/Medium?h=280">

                                        <div class="ribbon ribbon-top-left">
                                            <span>Best of</span>
                                        </div>

                                            <div class="energy-consumption epc-water-mark FlandersD"></div>
                                </div>
                            </a>



#seems this is for the ones under project having multiple houses 
<article itemscope="" itemtype="http://schema.org/Place" data-url="https://immovlan.be/en/projectdetail/25923-7222421" class="list-view-item mb-3 card card-border ">
                <div class="row">
                    <div class="col-lg-5 card-image project">


                        <div class="media-pic">
                            <a href="https://immovlan.be/en/projectdetail/25923-7222421" title="  Gent (RBU57093)" rel="nofollow">
                                <div class="media-pic-wrapper">
                                    <img class="centered-and-cropped ls-is-cached lazyloaded" alt="  Gent (RBU57093)" title="  Gent (RBU57093)" src="https://api-image.immovlan.be/v1/project/25923-7222421/image/889520bd09d24f20952c0cb0f00fb8d9.jpg/webp/Medium?h=280" data-src="https://api-image.immovlan.be/v1/project/25923-7222421/image/889520bd09d24f20952c0cb0f00fb8d9.jpg/webp/Medium?h=280" itemprop="photo" content="https://api-image.immovlan.be/v1/project/25923-7222421/image/889520bd09d24f20952c0cb0f00fb8d9.jpg/webp/Medium?h=280">

                                        <div class="ribbon ribbon-top-left is-update">
                                            <span>Updated</span>
                                        </div>

                                </div>
                            </a>
                        </div>



                    </div>
                    <div class="col-lg-7 pl-lg-0">
                        <div class="card-body position-relative project">
                            <div class="clearfix position-relative">
                                <div class="float-right text-right mt-1">


                                </div>
                                <div class="long-and-truncated">
                                    <h2 class="card-title ellipsis pr-2 mt-1 mb-0" itemprop="name">

                                            <a href="https://immovlan.be/en/projectdetail/25923-7222421" title="Project: Bloei" rel="nofollow">Project: Bloei</a>

                                    </h2>
                                </div>
                                <p class="mb-1 wp-75" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress">
                                    <span itemprop="postalCode">9000</span>
                                    <span itemprop="addressLocality">Gent</span>
                                </p>

                                    <span class="prefix grey-text text-right"></span>
                                    <strong class="list-item-price">250 000 € - 489 000 €</strong>

                            </div>

                            <p class="list-item-description mb-3">
                                <span itemprop="description"><strong></strong>Our website only shows a small part of what's on offer in the “Bloei” project. The last four apartments in Bloei Phase 1 will be finished in 2025 and will be ready to move into right away. This means that the new residents can enjoy their new home straight away, without any further renovations or refurbishments. Feel free to contact us for an overview of all available properties in Bloei and to visit the show apartment. Bloei stands for greenery, accessibility, and affordability in Ghent, with nature as a common thread running through it, located between a new sustainable park and an existing forest. Bloei is being built between Ghent-Sint-Pieters, The Loop, and AZ Maria Middelares. The city of Ghent will be investing in this project area in the coming years to create a new and vibrant district. The Project The Bloei residential project, spread over several buildings along the Vossenbos, consists of a mix of 1, 2, 3, and 4-bedroom apartments, all with a spacious terrace. Bloei's large windows draw light in, while you enjoy an open view of the park or forest, with privacy preserved. The Bloei apartments are practically laid out so that you can make optimal use of all the space and furnish it according to your own style and wishes. Thanks to stylish and high-quality finishes, the Bloei residential units have an attractive appearance. In phase 2, we offer you the Marjolein (4) building and the Steenanjer (5). Sustainability All Bloei residential units are nearly energy neutral (BEN) and fully comply with stricter energy consumption requirements. Among other things, solar panels and heat pumps are used. The Bloei park, which is currently under construction, is also dedicated to biodiversity. Optimally designed apartments with spacious terraces, a safe living environment, and multiple meeting places ensure that both young and old feel at home in “Bloei.” Location With a mobiscore of 8.9/10, Bloei is perfectly located near the Gent-Sint-Pieters station and the main roads to and from Ghent. The center of Ghent—with all its shops, cultural sites, and cozy restaurants—is easily accessible by bike and public transport. You can also easily reach Bloei by bike thanks to the multiple bike routes. A beautiful new apartment or a smart investment? Bloei has it all! Fixed and non-revisable price. Make your appointment now with Anthony on ### for detailed information.&nbsp;<a href="https://immovlan.be/en/projectdetail/25923-7222421" rel="nofollow">Want to know more?</a></span>
                            </p>

                                <div class="list-item-details owner-logo-space">
                                    <div class="d-flex justify-content-start flex-wrap mx-1">
                                            <span class="property-highlight margin-bottom-05 margin-right-05">
                                                <strong>59 - 134</strong>
                                                m²
                                            </span>
                                            <span class="property-highlight margin-bottom-05 margin-right-05">
                                                <strong>1</strong>
                                                Bathroom(s)
                                            </span>
                                            <span class="property-highlight margin-bottom-05 margin-right-05">
                                                <strong>10</strong>
                                                unit(s)
                                            </span>
                                    </div>
                                </div>

                                <div class="owner-logo">
                                        <div class="owner-logo-url" data-value-url="/tracking/prouser/logo-click/goto?proUserId=32719&amp;language=en" data-value-id="32719">
                                            <img src="https://api-image.immovlan.be/v1/prouser/01108793/logo/English/webp/Small" alt="Immo Da Vinci" title="Immo Da Vinci">
                                        </div>
                                </div>

                        </div>
                        
                        <!--DESKTOP-->
                            <div class="btn-group d-none d-lg-flex">
                                    <div class="button button-primary mt-auto text-capitalize flex-button btn-contact-project" data-value-id="25923-7222421">contact</div>
                                <a class="button button-secondary mt-auto ml-2 text-capitalize flex-button" href="https://immovlan.be/en/projectdetail/25923-7222421" rel="nofollow">details</a>
                            </div>                            

                        <!--MOBILE-->
                            <div class="btn-group d-flex d-lg-none">
                                    <div class="button button-primary button-with-icon button-no-radius mt-auto flex-button btn-contact-project" data-value-id="25923-7222421" aria-label="Contact by email">
                                        <i class="mi mi-mail mi-white"></i> <span class="text-truncate">Email</span>
                                    </div>

                                    <div class="button button-lg button-secondary button-with-icon button-no-radius mt-auto flex-button contact-by-phone-button" data-value-id="25923-7222421" data-value-property-id="10390773" data-value-project-id="25923-7222421" data-value-prouser-id="32719" data-value-prouser-types="estateAgents" data-value-prouser-type="estateAgents">
                                        <i class="mi mi-phone mi-primary"></i> <span class="text-truncate">Call</span>
                                    </div>
                            </div>                            

                        <meta itemprop="numberOfBedrooms">
                        <meta itemprop="numberOfBathroomsTotal">
                    </div>
                </div>
            </article>

next ................................;;


houses only ..................................

<a href="https://immovlan.be/en/detail/residence/for-sale/9000/gent/rbu64213" title="Residence for sale Gent (RBU64213)" rel="nofollow">

#########++++++++++++++++++++++++ pattern of the herf++++++++++++++++++++
ttps://immovlan.be/en/detail/[property_type]/for-sale/[postal_code]/[city]/[property_id]


In [ ]:

#Hamideh-url scraping(attempts)
#######################################################################
 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
import time
import csv

def collect_property_links(property_type, region_url, num_pages=50, output_file="property_links.csv"):
    """
    Collects property links from Immovlan for a given property type and region.
    
    Args:
        property_type (str): "house" or "apartment"
        region_url (str): Base URL of the region search results (e.g., Wallonia)
        num_pages (int): Number of pages to scrape
        output_file (str): CSV filename to save links
    """
    # ---- 1. Setup Selenium ----
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)

    all_links = []

    try:
        # ---- 2. Loop through the pages ----
        for page in range(1, num_pages + 1):
            
            #url = f"{region_url}?page={page}"
            
            #######################""
            
            if "?" in region_url:        #we added this part, thinkng maybe the problem is the ?
                url = f"{region_url}&page={page}"
            else:
                url = f"{region_url}?page={page}"
            ##########################
            driver.get(url)

            ###########################

            #the cookie based on the advice        #aaded cookies: thought maybe the cookies is the probem
            # ---- Handle cookie consent automatically ----
            try:
            # Wait for the "Agree and close" button to be clickable
                accept_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                )
                accept_button.click()
                print("Cookies accepted.")
            except Exception as e:
                print("No cookie popup found or could not click the accept button:", e)
    
# ---- End cookie handling ----


            ####################################                 #added this part thinking maybe the time of loading is an issue
            # --- 2. Wait for property links to appear ---
            try:
            # wait up to 10 seconds for at least 1 property link to appear
                WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, f"a[href*='/en/detail/house/']"))
                )
                print("Listings loaded.")
            except:
                print("Listings did not load in time.")


            ###########################################
            #time.sleep(3)  # wait for page to load
            
            # ---- 3. Find all links of the specified property type ----
            links = driver.find_elements(By.CSS_SELECTOR, f"a[href*='/en/detail/{property_type}/']")
            page_links = [link.get_attribute("href") for link in links if link.get_attribute("href")]
            all_links.extend(page_links)

            print(f"Page {page} scraped, found {len(page_links)} new links, total so far: {len(all_links)}")

        # ---- 4. Remove duplicates ----
        all_links = list(set(all_links))

        # ---- 5. Save to CSV ----
        with open(output_file, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([f"{property_type}_url"])
            for link in all_links:
                writer.writerow([link])
        
        print(f"✅ Done! Saved {len(all_links)} {property_type} links to {output_file}")

    finally:
        driver.quit()






        # URL for houses in Wallonia
east_flanders = "https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house&provinces=east-flanders&noindex=1"

# Call the function
collect_property_links(
    property_type="house", 
    region_url=east_flanders, 
    num_pages=5, 
    output_file="east_flanders_houses.csv"
)

Cookies accepted.
Listings did not load in time.
Page 1 scraped, found 0 new links, total so far: 0
No cookie popup found or could not click the accept button: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:202:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:555:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Listings did not load in time.
Page 2 scraped, found 0 new links, total so far: 0
No cookie popup found or could not click the accept button: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:202:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:555:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Listings did not load in time.
Page 3 scraped, found 0 new links, total so f

In [14]:

####This is the first version of code to find urls for the houses in Belgium: (Welde's task)


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
import time
import csv

# --------------------------
# 1. Setup
# --------------------------
options = Options()
options.add_argument("--headless")  # run without opening the browser window

service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)

# --------------------------
# 2. Open the listing page (example: apartments for sale in Belgium)
# --------------------------
url = "https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house&noindex=1"
driver.get(url)

# give page time to load fully (especially dynamic parts)
time.sleep(5)

# --------------------------
# 3. Find all apartment links
# --------------------------
links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/en/detail/apartment/']")
apartment_urls = [link.get_attribute("href") for link in links if link.get_attribute("href")]

# remove duplicates
apartment_urls = list(set(apartment_urls))

print(f"Found {len(apartment_urls)} apartment links.")

# --------------------------
# 4. Save the links to a CSV file
# --------------------------
with open("apartment_links.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["apartment_url"])
    for link in apartment_urls:
        writer.writerow([link])

print("✅ Saved all apartment links to apartment_links.csv")

# --------------------------
# 5. Close the browser
# --------------------------
driver.quit()




#Let's call the function

# URL for houses in Wallonia
wallonia_url = "https://immovlan.be/en/real-estate/house/for-sale/wallonia"

# Call the function
collect_property_links(
    property_type="house", 
    region_url=wallonia_url, 
    num_pages=10, 
    output_file="wallonia_houses.csv"
)


Found 0 apartment links.
✅ Saved all apartment links to apartment_links.csv


NameError: name 'collect_property_links' is not defined

In [1]:
#The welde code from his github

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from webdriver_manager.firefox import GeckoDriverManager
import csv
import time

# -----------------------
# Handle cookies popup
# -----------------------
def handle_cookies(driver, timeout=10):
    selectors = [
        "#onetrust-accept-btn-handler",
        "#didomi-notice-agree-button",
    ]
    for selector in selectors:
        try:
            cookie_btn = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            try:
                cookie_btn.click()
                print(f"Cookies accepted ({selector})")
                return
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
                time.sleep(0.5)
                cookie_btn.click()
                print(f"Cookies accepted after scrolling ({selector})")
                return
        except TimeoutException:
            continue
    print("No cookie popup found.")

# -----------------------
# Collect apartment URLs with precise filter
# -----------------------
def collect_apartment_urls(search_url, output_file="apartment_urls.csv"):
    """
    Collect apartment URLs based on precise HTML structure.
    """
    options = Options()
    options.headless = True
    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)
    all_urls = set()

    try:
        driver.get(search_url)
        handle_cookies(driver)
        time.sleep(2)

        while True:
            print("Scraping current page for apartment URLs...")

            # Wait for property links
            try:
                WebDriverWait(driver, 20).until(
                    EC.presence_of_all_elements_located(
                        (By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
                    )
                )
            except TimeoutException:
                print("No property links found on this page. Skipping...")
                continue

            # Scroll for lazy loading
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            # Collect apartment links only
            links = driver.find_elements(By.CSS_SELECTOR, "article.list-view-item h2.card-title a")
            for link in links:
                href = link.get_attribute("href")
                # Only include apartment URLs
                if href and "/apartment/" in href:
                    all_urls.add(href.split("?")[0])

            print(f"Found {len(links)} links on this page (total apartments so far: {len(all_urls)})")

            # Pagination: next page
            try:
                next_button = driver.find_element(
                    By.XPATH, "//a[contains(@class,'pagination__next') and not(contains(@class,'disabled'))]"
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(1)
                next_button.click()
                time.sleep(3)
            except:
                print("Reached the last page.")
                break

    finally:
        driver.quit()

    # Save URLs to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["URL"])
        for u in sorted(all_urls):
            writer.writerow([u])

    print(f"Saved {len(all_urls)} unique apartment URLs to '{output_file}'")
    return list(sorted(all_urls))


# -----------------------
# Example usage
# -----------------------
if __name__ == "__main__":
    search_url = (
        "https://immovlan.be/en/real-estate?"
        "transactiontypes=for-sale,in-public-sale&propertytypes=apartment&municipals=gent&noindex=1"
    )
    apartment_urls = collect_apartment_urls(search_url)
    print(f"Total collected apartment URLs: {len(apartment_urls)}")
    for u in apartment_urls[:]:
        print(u)

No cookie popup found.
Scraping current page for apartment URLs...
Found 20 links on this page (total apartments so far: 11)
Reached the last page.
Saved 11 unique apartment URLs to 'apartment_urls.csv'
Total collected apartment URLs: 11
https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu58960
https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu59338
https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu64759
https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu65487
https://immovlan.be/en/detail/apartment/for-sale/9000/gent/vbd48455
https://immovlan.be/en/detail/apartment/for-sale/9032/wondelgem/rbu64897
https://immovlan.be/en/detail/apartment/for-sale/9032/wondelgem/rbu64900
https://immovlan.be/en/detail/apartment/for-sale/9032/wondelgem/rbu64901
https://immovlan.be/en/detail/apartment/for-sale/9032/wondelgem/rbu64902
https://immovlan.be/en/detail/apartment/for-sale/9050/ledeberg/rbu64643
https://immovlan.be/en/detail/apartment/for-sale/9051/sint